In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
from surprise import Reader, Dataset

In [4]:
from surprise import Dataset, NMF
from surprise.model_selection import cross_validate


# Load the data
reader = Reader()
ratings = pd.read_csv('/kaggle/input/the-movies-dataset/ratings_small.csv')
ratings.head()
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

##load algo
algo = NMF()

# Run 5-fold cross-validation and print results
cross_validate(algo, data, measures=["RMSE", "MAE"], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9491  0.9502  0.9479  0.9451  0.9518  0.9488  0.0023  
MAE (testset)     0.7284  0.7320  0.7290  0.7275  0.7287  0.7291  0.0015  
Fit time          4.46    4.48    4.48    4.48    4.49    4.48    0.01    
Test time         0.35    0.19    0.34    0.19    0.19    0.25    0.07    


{'test_rmse': array([0.94906087, 0.95020451, 0.94787538, 0.94507193, 0.95179312]),
 'test_mae': array([0.72842346, 0.7319714 , 0.72901173, 0.72745887, 0.72871655]),
 'fit_time': (4.464179754257202,
  4.476733922958374,
  4.477855682373047,
  4.48350191116333,
  4.48835825920105),
 'test_time': (0.347179651260376,
  0.19142889976501465,
  0.33754587173461914,
  0.19238567352294922,
  0.1894972324371338)}

In [6]:
# data['userId']

In [7]:
from surprise import accuracy
from surprise.model_selection import train_test_split

# sample random trainset and testset
# test set is made of 25% of the ratings.
trainset, testset = train_test_split(data, test_size=0.25, random_state = 42)

# Train the algorithm on the trainset, and predict ratings for the testset
algo.fit(trainset)
predictions = algo.test(testset)

# Then compute RMSE
accuracy.rmse(predictions)

RMSE: 0.9498


0.9498037180461806

In [8]:
# testset

In [9]:
predictions

[Prediction(uid=646, iid=2012, r_ui=5.0, est=3.536366494435722, details={'was_impossible': False}),
 Prediction(uid=652, iid=100017, r_ui=3.0, est=3.5467847952748555, details={'was_impossible': True, 'reason': 'User and item are unknown.'}),
 Prediction(uid=30, iid=2871, r_ui=4.0, est=4.385797235564041, details={'was_impossible': False}),
 Prediction(uid=523, iid=1265, r_ui=5.0, est=4.087544305015918, details={'was_impossible': False}),
 Prediction(uid=24, iid=165, r_ui=4.0, est=3.7713043120109964, details={'was_impossible': False}),
 Prediction(uid=23, iid=2011, r_ui=4.0, est=3.5110955220604096, details={'was_impossible': False}),
 Prediction(uid=157, iid=81417, r_ui=3.5, est=3.5467847952748555, details={'was_impossible': True, 'reason': 'User and item are unknown.'}),
 Prediction(uid=220, iid=3108, r_ui=3.0, est=3.590606785809366, details={'was_impossible': False}),
 Prediction(uid=103, iid=2959, r_ui=5.0, est=4.147116992935093, details={'was_impossible': False}),
 Prediction(uid=646

In [ ]:
# Than predict ratings for all pairs (u, i) that are NOT in the training set.
anti_testset = trainset.build_anti_testset()
predictions = algo.test(anti_testset)

In [15]:
pd.DataFrame(predictions).sort_values(by='est')[::-1]

,uid,iid,r_ui,est,details
17866,637,858,5.0,5.0,{'was_impossible': False}
10153,543,1356,5.0,5.0,{'was_impossible': False}
199,299,1361,4.5,5.0,{'was_impossible': False}
18830,481,3310,5.0,5.0,{'was_impossible': False}
20416,558,4848,5.0,5.0,{'was_impossible': False}
...,...,...,...,...,...
17519,247,3440,2.0,1.0,{'was_impossible': False}
21081,452,5832,1.0,1.0,{'was_impossible': False}
10197,456,130448,0.5,1.0,{'was_impossible': False}
9134,119,3895,2.0,1.0,{'was_impossible': False}


In [ ]:
# predictions

In [ ]:
from collections import defaultdict
def get_top_n(predictions, n=10):
    """Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    """

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

In [ ]:
top_n = get_top_n(predictions, n=10)

# Print the recommended items for each user
for uid, user_ratings in top_n.items():
    print(uid, [iid for (iid, _) in user_ratings])

In [ ]:
top_n[14]


In [ ]:
data

## Now we define functions to easily use it 

In [14]:
# collaborative algorithm

import numpy as np
import pandas as pd



from surprise import Dataset, Reader
from surprise import NMF, SVD
# from surprise.model_selection import train_test_split
from surprise import accuracy
from surprise.dataset import DatasetAutoFolds

from collections import defaultdict

data = pd.read_csv('/kaggle/input/the-movies-dataset/ratings_small.csv')
trainset, testset = train_test_split(data, test_size=0.25, random_state = 42)

class CollaborativeFiltering:
    def __init__(trainset: pd.DataFrame, testset: pd.DataFrame, algorithm=None):
        self.trainset = trainset
        self.testset = testset
        self.algorithm = algorithm
        self.predictions = None
        
    def load_data():
        reader = Reader()
  
        train = Dataset.load_from_df(trainset[['userId', 'movieId', 'rating']], reader)
        test = Dataset.load_from_df(testset[['userId', 'movieId', 'rating']], reader)
        
        self.trainset = train
        self.testset = test
        
    def fit_predict():
    
        self.algorithm.fit(self.trainset)
        predictions_test = self.algorithm.test(self.testset)
        self.predictions = predictions_test
        
        return accuracy.rmse(predictions_test)





In [7]:
top_n[14]
# user 14, suggest 10 movies 121231,...,3320 with rating estimated 5,..., 4.777

NameError: name 'top_n' is not defined

In [ ]:
## Example

top_n = collaborative_filtering(input='/kaggle/input/the-movies-dataset/ratings_small.csv', algorithm=SVD())

# Print the recommended items for each user
# for uid, user_ratings in top_n.items():
#     print(uid, [iid for (iid, _) in user_ratings])

In [ ]:
top_n[14]
# user 14, suggest 10 movies 1172,...,527 with rating estimated 4.366,..., 4.246

In [ ]:
## Following the git repository

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
data = pd.read_csv('/kaggle/input/the-movies-dataset/ratings_small.csv')
trainset, testset = train_test_split(data, test_size=0.25, random_state = 42, stratify=data['userId'])

trainset

,userId,movieId,rating,timestamp
59679,431,2863,4.0,1165548515
85061,571,7173,2.0,1334343358
12058,77,223,4.5,1163004353
86645,580,1032,4.0,1165291033
93715,624,1221,5.0,1019124147
...,...,...,...,...
79676,547,5810,3.0,1415444349
57656,418,1835,4.0,1132180632
444,5,33679,4.0,1163374517
48890,358,905,5.0,957479957


In [2]:

import numpy as np
import pandas as pd



from surprise import Dataset, Reader
from surprise import NMF, SVD
# from surprise.model_selection import train_test_split
from surprise import accuracy
from surprise.dataset import DatasetAutoFolds

from collections import defaultdict

In [12]:
from surprise import SVD, BaselineOnly, SVDpp, NMF, SlopeOne, CoClustering, Reader

In [35]:
class CollaborativeFiltering:
    def __init__(self, algorithm=None):
        self.trainset = None
        self.testset = None
        self.algorithm = algorithm
        self.predictions = None
        
    def load_data(self, trainset, testset):
        reader = Reader()
  
        train = Dataset.load_from_df(trainset[['userId', 'movieId', 'rating']], reader)
        test = Dataset.load_from_df(testset[['userId', 'movieId', 'rating']], reader)
        
        self.trainset = train.build_full_trainset()
        full_testset = test.build_full_trainset()
        self.testset = full_testset.build_testset()
        
    def fit_predict(self):
    
        self.algorithm.fit(self.trainset)
        predictions_test = self.algorithm.test(self.testset)
        self.predictions = predictions_test
        
        return accuracy.rmse(predictions_test), accuracy.mae(predictions_test)



In [36]:
svd = CollaborativeFiltering(SVD())
svd.load_data(trainset, testset)
svd.fit_predict()

RMSE: 0.9059
MAE:  0.6979


(0.9059170329875486, 0.6978924637083077)

In [37]:
nmf = CollaborativeFiltering(NMF())
nmf.load_data(trainset, testset)
nmf.fit_predict()



RMSE: 0.9543
MAE:  0.7340


(0.954277116291718, 0.7340400133995559)

In [38]:
df = pd.DataFrame(nmf.predictions).drop(columns='details')

df.columns = ['uid', 'iid', 'r_ui', 'nmf_rating']
df.head()

,uid,iid,r_ui,nmf_rating
0,302,593,5.0,3.908355
1,302,587,3.0,3.187583
2,302,595,4.0,3.423041
3,302,457,4.0,3.652733
4,302,256,3.0,2.945739


In [39]:
svd = CollaborativeFiltering(SVDpp())
svd.load_data(trainset, testset)
svd.fit_predict()

RMSE: 0.8929
MAE:  0.6839


(0.8928898996286371, 0.6839025472454057)

In [40]:
base = CollaborativeFiltering(BaselineOnly())
base.load_data(trainset, testset)
base.fit_predict()

Estimating biases using als...
RMSE: 0.8995
MAE:  0.6951


(0.8994895068779108, 0.6950960226235026)

In [41]:
slope_one = CollaborativeFiltering(SlopeOne())
slope_one.load_data(trainset, testset)
slope_one.fit_predict()

RMSE: 0.9357
MAE:  0.7176


(0.9356594841055151, 0.717627648179268)

In [54]:
df['slope_one_rating'] = pd.DataFrame(slope_one.predictions)['est']

In [42]:
from surprise.prediction_algorithms import KNNBaseline, KNNBasic, KNNWithMeans, KNNWithZScore

In [43]:
svd = CollaborativeFiltering(KNNBaseline())
svd.load_data(trainset, testset)
svd.fit_predict()

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9035
MAE:  0.6917


(0.9035103550736302, 0.6916821174221489)

In [44]:
KNN_basic = CollaborativeFiltering(KNNBasic())
KNN_basic.load_data(trainset, testset)
KNN_basic.fit_predict()

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9745
MAE:  0.7501


(0.9745084803589178, 0.7501069146686726)

In [52]:
df['knn_basic_rating'] = pd.DataFrame(KNN_basic.predictions)['est']

In [46]:
KNN_means = CollaborativeFiltering(KNNWithMeans())
KNN_means.load_data(trainset, testset)
KNN_means.fit_predict()

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9260
MAE:  0.7089


(0.9260352128943168, 0.7089121531916549)

In [47]:
KNN_Z = CollaborativeFiltering(KNNWithZScore())
KNN_Z.load_data(trainset, testset)
KNN_Z.fit_predict()

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9246
MAE:  0.7036


(0.924571110767598, 0.7035651215990303)

In [55]:
df

,uid,iid,r_ui,nmf_rating,knn_basic_rating,slope_one,slope_one_rating
0,302,593,5.0,3.908355,4.071692,4.081150,4.081150
1,302,587,3.0,3.187583,3.486138,3.214561,3.214561
2,302,595,4.0,3.423041,3.884976,3.559051,3.559051
3,302,457,4.0,3.652733,3.933139,3.861834,3.861834
4,302,256,3.0,2.945739,2.908185,2.642205,2.642205
...,...,...,...,...,...,...,...
24996,498,5459,3.0,2.550204,3.053953,2.522159,2.522159
24997,498,588,4.5,3.601806,3.372992,3.060440,3.060440
24998,498,520,3.0,2.963942,2.963016,2.663275,2.663275
24999,498,1544,4.0,2.117669,3.006170,2.204511,2.204511


In [56]:
import math

In [60]:

# svd_wt = 0.05
# knn_wt = 0.6
# svdpp_wt = 0.4
slopeone_wt = 0.4
nmf_wt = 0.6
knn_basic_wt = 0

In [59]:

rmse = ((df.r_ui - df.knn_basic_rating) ** 2).mean() ** .5
print(rmse)
mae = (((df.r_ui - df.knn_basic_rating) ** 2) ** .5).mean()
print(mae)

0.9745084803589178
0.7501069146686726


In [62]:
T = df.shape[0]
print(T)

25001


In [64]:

sqr_sum = 0
abs_sum = 0

for ind, row in df.iterrows():
    org_r = row['r_ui']
#   pred_r = svd_wt*row['svd_rating'] + knn_wt*row['knn_basic_rating'] + svdpp_wt*row['svdpp_rating'] + slopeone_wt*row['slopeone_rating'] + baseline_wt*row['baseline_rating']
#   diff = np.abs(org_r - pred_r)

    pred_r = knn_basic_wt*row['knn_basic_rating'] + slopeone_wt*row['slope_one_rating'] + nmf_wt*row['nmf_rating']
    diff = np.abs(org_r - pred_r)
  # print(diff)
    abs_sum += diff
    sqr_sum += diff**2

rmse = np.sqrt(sqr_sum/T)
print("RMSE", rmse)
mae = abs_sum/T
print("MAE", mae)

RMSE 0.9275877940427071
MAE 0.7135741775201034
